<a href="https://colab.research.google.com/github/LYZAcell/data_study/blob/main/250312_10%E1%84%80%E1%85%B5_%E1%84%83%E1%85%A6%E1%84%8B%E1%85%B5%E1%84%90%E1%85%A5%E1%84%87%E1%85%AE%E1%86%AB%E1%84%89%E1%85%A5%E1%86%A8%E1%84%89%E1%85%B5%E1%86%AF%E1%84%8C%E1%85%A5%E1%86%AB%E1%84%87%E1%85%A1%E1%86%AB_GA4_%E1%84%83%E1%85%A6%E1%84%8B%E1%85%B5%E1%84%90%E1%85%A5_%E1%84%8B%E1%85%B5%E1%84%92%E1%85%A2(1)_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('2019-Dec.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3533286 entries, 0 to 3533285
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 242.6+ MB


In [ ]:
df.isna().sum()

event_time             0
event_type             0
product_id             0
category_id            0
category_code    3474821
brand            1510289
price                  0
user_id                0
user_session         779
dtype: int64

In [ ]:
## 데이터 샘플로 뽑아서 분석 진행
df_sample =df.sample(n=50000, random_state=111)
print(df_sample)

                      event_time        event_type  product_id  \
1141402  2019-12-09 19:04:29 UTC              view     5814516   
2826120  2019-12-23 12:44:51 UTC              view     5915004   
354426   2019-12-03 16:25:54 UTC              view     5877456   
3089023  2019-12-25 19:06:15 UTC              cart     5608703   
1762400  2019-12-14 11:41:25 UTC              view     5769914   
...                          ...               ...         ...   
954138   2019-12-08 14:29:44 UTC  remove_from_cart     5695961   
1363136  2019-12-11 10:17:56 UTC              cart     5800038   
3033914  2019-12-25 10:03:55 UTC          purchase     5629984   
3455329  2019-12-30 08:41:10 UTC          purchase     5851607   
2220662  2019-12-17 21:18:58 UTC              cart     5678044   

                 category_id category_code     brand   price    user_id  \
1141402  1487580013539623112           NaN       NaN   13.49  247790217   
2826120  1487580008028308447           NaN  babyliss   96

In [ ]:
##Event가 무엇이 있는지?
##퍼널이 닫혀있는 가정
df_sample.event_type.value_counts()

event_type
view                24451
cart                13178
remove_from_cart     9310
purchase             3061
Name: count, dtype: int64

In [ ]:
## 퍼널에 대한 비교도 보고싶고
## Event별 제품이 다를까~?
## 제품을 다량으로 구매하는 경우, 단품으로 구매하는 경우 -> 제품을 구매하는 어떤 범위가 어느정도인가?
## 어떤 제품들끼리 잘 묶여서 구매가 되고 있는건가?!
## 우리가 추천할 영역의 값이 무엇을 기준으로 해야 하나? -> product_id 것을 추천
## product_id가 얼마나 영역의 커버리지할 수 있는 범위
## 추천시스템을 구현할 때
## 개인화추천시스템 -> 그전에 이미 단순하게도 충분히 구현할 수 있고, 꼭 개인화가 아니더라도 이벤트에 따라서 또는 실제 단순한 로직으로도 매출에 대해서 영향을 줄 수 있다.
## 단순한 로직으로 추천
## 가장 많이 구매한 것, 가장 같이 잘 팔리는 것 (동시구매 co-occurence), event 별 추천, 제품 또는 검색어 기반으로 추천도 달라질 수 있다.


## 가설 1 . Event별 제품의 베스트가 차이가 클까?

top_viewed=df_sample[df_sample['event_type']=='view']['product_id'].value_counts().head(5)
top_carted=df_sample[df_sample['event_type']=='cart']['product_id'].value_counts().head(5)
top_purchased=df_sample[df_sample['event_type']=='purchase']['product_id'].value_counts().head(5)

In [ ]:
print('top_viewed',top_viewed)
print('top_carted',top_carted)
print('top_purchased',top_purchased)

top_viewed product_id
5809910    353
5909810    115
5877454     94
5809912     89
5886282     81
Name: count, dtype: int64
top_carted product_id
5809910    90
5700037    80
5802432    75
5854897    52
5815662    47
Name: count, dtype: int64
top_purchased product_id
5809910    22
5700037    12
5854897    11
5528035    11
5809912    11
Name: count, dtype: int64


In [ ]:
## 퍼널에 대한 확인

# 시계열로 변환
df_sample['event_time'] = pd.to_datetime(df_sample['event_time'])

## event_time 일자로 뽑으려면
df_sample['date']=df_sample['event_time'].dt.date

## 우리가 지금 퍼널을 카운팅하기 하기 위해서는 어떤 기준으로 가야할까?
## user_id 카운팅
daily_funnel_cohort=df_sample.groupby(['date','event_type'])['user_id'].nunique().unstack(fill_value=0)
## cohort 컬럼 정리
daily_funnel_cohort=daily_funnel_cohort[['view','cart','purchase']]

## 전환율 계산
daily_funnel_cohort['view->cart(%)'] = (daily_funnel_cohort['cart']/daily_funnel_cohort['view']*100).round(2)
daily_funnel_cohort['cart->purchase(%)'] = (daily_funnel_cohort['purchase']/daily_funnel_cohort['cart']*100).round(2)
daily_funnel_cohort['view->purchase(%)'] = (daily_funnel_cohort['purchase']/daily_funnel_cohort['view']*100).round(2)



In [ ]:
daily_funnel_cohort

event_type,view,cart,purchase,view->cart(%),cart->purchase(%),view->purchase(%)
date,,,,,,
2019-12-01,766,401,89,52.35,22.19,11.62
2019-12-02,825,413,89,50.06,21.55,10.79
2019-12-03,805,460,115,57.14,25.00,14.29
2019-12-04,779,439,103,56.35,23.46,13.22
2019-12-05,766,393,122,51.31,31.04,15.93
2019-12-06,719,404,105,56.19,25.99,14.60
2019-12-07,696,381,80,54.74,21.00,11.49
2019-12-08,753,403,83,53.52,20.60,11.02
2019-12-09,936,484,102,51.71,21.07,10.90


### 필수과제1
- 2019년 12월, 2019 10월, 11월 데이터도 샘플링하셔서 (100,000만 개 이상으로 샘플링해서)
- 필수과제1.1. 월별 퍼널에 대한 분석이 필요하다. event에 대한 유저들의 퍼널율 계산하고
- 필수과제1.2. 요일 지수를 확인해야 하므로, 요일 지수에 따른 시각화와 인사이트 정리
- 필수과제1.3. 시간에 따른 인사이트 정리 필요 -> 시간에 따른 시각화와 인사이트 정리
- 필수과제1.4. 베스트 제품의 이벤트별 달라지는가? -> 월별에 따른 이벤트별 베스트 제품이 달라지는가? 인사이트 정리
----
- 추천하기 전 제품들에 대한 월별, 시간별, 요일별 트렌드가 존재하는지를 살펴보자!
- 총 4개의 파일로 ipynb 만들어서 노션에 업로드 하시면 됩니다.

In [ ]:
df_sample.groupby(['date','event_type'])['user_id'].nunique().unstack(fill_value=0)

event_type,cart,purchase,remove_from_cart,view
date,,,,
2019-12-01,401,89,282,766
2019-12-02,413,89,299,825
2019-12-03,460,115,295,805
2019-12-04,439,103,291,779
2019-12-05,393,122,274,766
2019-12-06,404,105,263,719
2019-12-07,381,80,218,696
2019-12-08,403,83,270,753
2019-12-09,484,102,327,936


In [ ]:
df_sample['event_time'].dt.date

1141402    2019-12-09
2826120    2019-12-23
354426     2019-12-03
3089023    2019-12-25
1762400    2019-12-14
              ...    
954138     2019-12-08
1363136    2019-12-11
3033914    2019-12-25
3455329    2019-12-30
2220662    2019-12-17
Name: event_time, Length: 50000, dtype: object

In [ ]:
df_sample

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,date
1141402,2019-12-09 19:04:29+00:00,view,5814516,1487580013539623112,NaN,NaN,13.49,247790217,a897a9a9-5ebe-46df-91a0-db71d9fda2f0,2019-12-09
2826120,2019-12-23 12:44:51+00:00,view,5915004,1487580008028308447,NaN,babyliss,96.51,514193964,5974f0cb-0916-469c-bf62-8843c7208959,2019-12-23
354426,2019-12-03 16:25:54+00:00,view,5877456,1487580006300255120,NaN,jessnail,122.22,580819864,f508ad9a-198e-43f7-9772-e659133f4025,2019-12-03
3089023,2019-12-25 19:06:15+00:00,cart,5608703,1487580005553668971,NaN,NaN,9.52,430430758,d9c35cd6-ce2b-445c-b467-4050a854f1c6,2019-12-25
1762400,2019-12-14 11:41:25+00:00,view,5769914,1487580013841613016,NaN,kapous,3.65,586788406,6248f4cb-e919-4251-afb7-3cac961d787c,2019-12-14
...,...,...,...,...,...,...,...,...,...,...
954138,2019-12-08 14:29:44+00:00,remove_from_cart,5695961,1487580007575323592,NaN,NaN,1.43,448126743,937f8855-e137-4192-9c3f-787bf692317a,2019-12-08
1363136,2019-12-11 10:17:56+00:00,cart,5800038,1487580005385896804,NaN,NaN,5.56,585244747,cb2505b0-7b37-4f15-b9a3-18cb4cd2999c,2019-12-11
3033914,2019-12-25 10:03:55+00:00,purchase,5629984,1487580005754995573,NaN,NaN,4.75,592260899,f6d34c77-c172-4714-be9e-873efe737346,2019-12-25
3455329,2019-12-30 08:41:10+00:00,purchase,5851607,2055161088059638328,NaN,irisk,1.75,563932069,5470ca45-c088-446b-8b6d-dfe7f68e81d4,2019-12-30
